# Libraries

In [69]:
import numpy as np
import pandas as pd
import pickle
import tqdm.auto as tqdm
from pprint import pprint
import collections

# Use restricted_wcc dataset (only courses from WCC, last five years, no summer)

In [18]:
WCC_PD_PATH = "../data/wcc_pd.pkl"

with open(WCC_PD_PATH, "rb") as file:
    raw_data = pickle.load(file)
    
raw_data

,acad_career,strm,stdnt_enrl_status,unt_taken,unt_billing,crse_grade_input,earn_credit,emplid,subject,catalog_nbr,crse_acad_org,sex,course_name
0,UG,1182.0,E,5.0,5.0,A,Y,$2a$15$.iQPCHeeuyLD3TIqJRk4j.LU0IjGYumSdFkAEUf...,MATH,51,MATH,M,MATH51
1,UG,1182.0,E,0.0,0.0,None,N,$2a$15$.iQPCHeeuyLD3TIqJRk4j.LU0IjGYumSdFkAEUf...,CHEM,31X,CHEMISTRY,M,CHEM31X
2,UG,1182.0,E,5.0,5.0,A,Y,$2a$15$.iQPCHeeuyLD3TIqJRk4j.LU0IjGYumSdFkAEUf...,CHEM,31X,CHEMISTRY,M,CHEM31X
3,UG,1182.0,E,0.0,0.0,None,N,$2a$15$.iQPCHeeuyLD3TIqJRk4j.LU0IjGYumSdFkAEUf...,MATH,51,MATH,M,MATH51
4,UG,1184.0,E,5.0,5.0,A,Y,$2a$15$.iQPCHeeuyLD3TIqJRk4j.LU0IjGYumSdFkAEUf...,CS,106X,COMPUTSCI,M,CS106X
...,...,...,...,...,...,...,...,...,...,...,...,...,...
784350,UG,1202.0,E,5.0,5.0,B,Y,ZZCaHx+CP6JrUW8AJ/N+m9uPlGbsOEaeCC8uU50l/iVr9d...,CHEM,31A,CHEMISTRY,F,CHEM31A
784351,UG,1202.0,E,0.0,0.0,None,N,ZZCaHx+CP6JrUW8AJ/N+m9uPlGbsOEaeCC8uU50l/iVr9d...,CHEM,31A,CHEMISTRY,F,CHEM31A
784352,UG,1204.0,E,5.0,5.0,None,Y,ZZCaHx+CP6JrUW8AJ/N+m9uPlGbsOEaeCC8uU50l/iVr9d...,PSYCH,1,PSYCHOLOGY,F,PSYCH1
784353,UG,1204.0,E,5.0,5.0,None,Y,ZZCaHx+CP6JrUW8AJ/N+m9uPlGbsOEaeCC8uU50l/iVr9d...,CHEM,31B,CHEMISTRY,F,CHEM31B


In [50]:
print(raw_data.loc[1], "\n")
print(raw_data.loc[2], "\n")
print(raw_data.loc[1] == raw_data.loc[2])

acad_career                                                         UG
strm                                                              1182
stdnt_enrl_status                                                    E
unt_taken                                                            0
unt_billing                                                          0
crse_grade_input                                                  None
earn_credit                                                          N
emplid               $2a$15$.iQPCHeeuyLD3TIqJRk4j.LU0IjGYumSdFkAEUf...
subject                                                           CHEM
catalog_nbr                                                        31X
crse_acad_org                                                CHEMISTRY
sex                                                                  M
course_name                                                    CHEM31X
Name: 1, dtype: object 

acad_career                                         

In [51]:
print(raw_data.loc[784350], "\n")
print(raw_data.loc[784351], "\n")
print(raw_data.loc[784350] == raw_data.loc[784351])

acad_career                                                         UG
strm                                                              1202
stdnt_enrl_status                                                    E
unt_taken                                                            5
unt_billing                                                          5
crse_grade_input                                                     B
earn_credit                                                          Y
emplid               ZZCaHx+CP6JrUW8AJ/N+m9uPlGbsOEaeCC8uU50l/iVr9d...
subject                                                           CHEM
catalog_nbr                                                        31A
crse_acad_org                                                CHEMISTRY
sex                                                                  F
course_name                                                    CHEM31A
Name: 784350, dtype: object 

acad_career                                    

In [4]:
all_courses = set(raw_data["course_name"])
all_courses

{'AA100',
 'AA200',
 'AA210A',
 'AA212',
 'AA214A',
 'AA214B',
 'AA228',
 'AA229',
 'AA240A',
 'AA240B',
 'AA241A',
 'AA241B',
 'AA241X',
 'AA242A',
 'AA256',
 'AA270',
 'AA271A',
 'AA272C',
 'AA279A',
 'AA279B',
 'AA280',
 'APPPHYS272',
 'APPPHYS273',
 'BIO101',
 'BIO104',
 'BIO107',
 'BIO115',
 'BIO144',
 'BIO153',
 'BIO156',
 'BIO158',
 'BIO173',
 'BIO188',
 'BIO189',
 'BIO234',
 'BIO239',
 'BIO256',
 'BIO258',
 'BIO41',
 'BIO42',
 'BIO43',
 'BIO44X',
 'BIO44Y',
 'BIOE101',
 'BIOE103',
 'BIOE103B',
 'BIOE123',
 'BIOE141A',
 'BIOE141B',
 'BIOE201C',
 'BIOE210',
 'BIOE211',
 'BIOE212',
 'BIOE244',
 'BIOE280',
 'BIOE300A',
 'BIOE301C',
 'BIOE311',
 'BIOE313',
 'BIOE355',
 'BIOE41',
 'BIOE42',
 'BIOE44',
 'BIOE454',
 'BIOHOPK163H',
 'BIOHOPK167H',
 'BIOHOPK263H',
 'BIOMEDIN156',
 'BIOMEDIN210',
 'BIOMEDIN212',
 'BIOMEDIN214',
 'BIOMEDIN215',
 'BIOMEDIN217',
 'BIOMEDIN224',
 'BIOMEDIN225',
 'BIOMEDIN256',
 'BIOMEDIN260',
 'BIOMEDIN371',
 'BIOPHYS311',
 'BIOPHYS371',
 'CEE101A',
 'CEE101B

Winter is 4, Spring is 6, Fall is 2, Summer is 8?
e.g. 1198 is 18-19, summer quarter. 1196 is 18-19, spring quarter

In [70]:
# def get_course_name (row):
#     return row["course_name"]

# def dropped_function (row):
#     if row['stdnt_enrl_status'] == 'D':
#         return 1
#     return 0

In [99]:
LETTER_GRADES = ["A+", "A", "A-", "B+", "B", "B-", "C+", "C", "C-", "D+", "D", "D-", "NP", "W"]

# Builds the feature matrix X and output vector y for a given course, using all other courses in the dataset
# as possible predictors. Only adds a column for another course if at least one student has taken that other
# course before the given course.
# Set gender = True to also add a prediction vector for female (0/1).
# Returns a dataframe with y appended to the right of X, split them off later in get_and_split_data().
############################################################################################################
# 2020-02-23 Initial model settings: 
# Predict success in a course (B+ or better) based on having passed other courses previously (D- or better)
# Incomplete data for strm 1204 (2019-2020 Winter)
# Y1GRADES = ["A+", "A", "A-", "B+"]
# X1GRADES = ["A+", "A", "A-", "B+", "B", "B-", 
#             "C+", "C", "C-", "D+", "D", "D-"]
# UNFINISHED_QUARTERS = ["1204"]

Y1GRADES = ["A+", "A", "A-", "B+"] # we encode these grades as y = 1, otherwise y = 0
X1GRADES = ["A+", "A", "A-", "B+", "B", "B-", 
            "C+", "C", "C-", "D+", "D", "D-"] # for previous courses, encode these grades as X_i = 1
UNFINISHED_QUARTERS = ["1204"] # unfinished quarter(s) if pulled during a quarter

def getStudentFeatures(course_name, gender = False):
    course_entries = raw_data.loc[(raw_data['course_name'] == course_name) & # all course entries that are about the certain course we want
                                  (raw_data['crse_grade_input'].isin(LETTER_GRADES)) & # Grade needs to be a letter
                                  (raw_data['stdnt_enrl_status'] != 'D') & # not dropped
                                  (~raw_data['strm'].isin(UNFINISHED_QUARTERS))] # we don't have outcomes for these
    students = course_entries.emplid # Students from the course entries
    students_prev_classes = raw_data.loc[raw_data['emplid'].isin(students)] # Other classes taken by these students
    possible_prev_classes = students_prev_classes['course_name'].unique().tolist() # List of courses taken by these students
    possible_prev_classes.append("strm") # For now add term to the data of when the course was taken
    if gender:
        possible_prev_classes.append("female") # Add gender
    possible_prev_classes.append("emplid") # Add emplid
    possible_prev_classes.append("y") # Add the target to the data. Will split it later
    a = np.zeros(shape = (len(course_entries), len(possible_prev_classes))) # all 0 initialized features matrix
    full_matrix = pd.DataFrame(a, columns = possible_prev_classes) # same as above, but in pandas to allow indexing by course name
    
    for counter, row_idx in enumerate(course_entries.index):
        print(counter, row_idx)

#     for counter, idx in enumerate(course_entries.index): # loop over all class entries
# #     for counter, idx in course_entries.iterrows():
# #         print("student: ", counter, " of ", len(course_entries))
#         grade = course_entries.loc[[idx]]['crse_grade_input']
#         gradeStr = grade.tolist()[0]
#         if (gradeStr in Y1GRADES):
#             full_matrix['y'][counter] = 1
#         # fill predictor matrix X
#         std_id = course_entries.loc[[idx]]['emplid'] # Get the student id
#         quarter = course_entries.loc[[idx]]['strm'] # Get the term
#         if gender:
#             female = course_entries.loc[[idx]]['sex'] # Get the student gender
#         student_courses = students_prev_classes.loc[students_prev_classes['emplid'] == std_id.tolist()[0]] # Get all courses the student took
#         student_courses = student_courses.loc[student_courses['strm'] < quarter.tolist()[0]] # Filter the courses to be only before the course of interest
#         if gender:
#             if (female.tolist()[0] == 'M'):
#                 full_matrix["female"][counter] = 0
#             else:
#                 full_matrix["female"][counter] = 1
#         full_matrix["strm"][counter] = quarter.tolist()[0]
#         full_matrix["emplid"][counter] = std_id.tolist()[0]
#         for j in student_courses.index:# Loop over the student's courses
#             course_name = student_courses.loc[[j]]['subject']+student_courses.loc[[j]]['catalog_nbr'] # Get the course name
#             grade = student_courses.loc[[j]]['crse_grade_input']
#             gradeStr = grade.tolist()[0]
#             if (gradeStr in X1GRADES):
#                 full_matrix.iloc[counter, full_matrix.columns.get_loc(course_name.tolist()[0])] = 1 # Set the course name in X to the new value (or if it was already passed keep the value)
# #             dev_break = True
    return full_matrix

In [100]:
#dataset includes a y column, which is the target. It also includes strm and emplid columns that should
#be dropped before running any prediction
dataset_110 = getStudentFeatures('CS110')
dataset_110

0 15
1 115
2 160
3 198
4 212
5 343
6 360
7 404
8 547
9 647
10 745
11 769
12 790
13 918
14 1221
15 1225
16 1560
17 1673
18 2904
19 2991
20 3528
21 3944
22 5513
23 7358
24 7713
25 8336
26 8564
27 8664
28 9413
29 10363
30 11418
31 11653
32 13845
33 13942
34 14281
35 14424
36 16657
37 17546
38 17610
39 17713
40 17955
41 18626
42 19093
43 20159
44 22244
45 22320
46 22928
47 23727
48 24427
49 24471
50 25796
51 27652
52 27657
53 28295
54 28882
55 29919
56 30013
57 31674
58 32016
59 33315
60 35135
61 36045
62 36225
63 36621
64 37172
65 40304
66 40503
67 40885
68 41251
69 41432
70 41866
71 41988
72 42338
73 43531
74 44473
75 45979
76 46663
77 46734
78 47565
79 47573
80 48429
81 48538
82 49300
83 49302
84 49346
85 49577
86 50161
87 50652
88 51571
89 52179
90 52834
91 53201
92 53203
93 53778
94 54531
95 55526
96 55953
97 56930
98 58036
99 58277
100 58283
101 59060
102 59104
103 59239
104 59944
105 59948
106 60825
107 61078
108 61084
109 61630
110 61901
111 62322
112 62395
113 62527
114 62532
115 

1387 747700
1388 747864
1389 748190
1390 748835
1391 749479
1392 749497
1393 749592
1394 749824
1395 749933
1396 750127
1397 750138
1398 750193
1399 750381
1400 751051
1401 751161
1402 751468
1403 752311
1404 752380
1405 752609
1406 752874
1407 753174
1408 753345
1409 753463
1410 753530
1411 753601
1412 753938
1413 753970
1414 754127
1415 754250
1416 754576
1417 754643
1418 754734
1419 755007
1420 755237
1421 755284
1422 755371
1423 755633
1424 755730
1425 756219
1426 756494
1427 756622
1428 756925
1429 757496
1430 757975
1431 758061
1432 758426
1433 759006
1434 759199
1435 759311
1436 759567
1437 759655
1438 759827
1439 760721
1440 760763
1441 761067
1442 761147
1443 761378
1444 761597
1445 761838
1446 761936
1447 762265
1448 762340
1449 762495
1450 762677
1451 763174
1452 763264
1453 763346
1454 763920
1455 764340
1456 764614
1457 764830
1458 765522
1459 765626
1460 765658
1461 765771
1462 765933
1463 766144
1464 766212
1465 766230
1466 766527
1467 766731
1468 766918
1469 767485
1470

,MATH51,CHEM31X,CS106X,CHEM33,PHIL1,CS103,CHEM35,CS109,CS161,CS107,...,PHYSICS231,PHIL251,BIO153,PHYSICS212,PHYSICS331,LINGUIST205A,MATH226,strm,emplid,y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1536,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
collections.Counter(dataset_110.y)

Counter({1.0: 669, 0.0: 870})

# Try sklearn logistic regression

In [9]:
from sklearn.linear_model import LogisticRegression

In [10]:
# Use the X predictor matrix and y outcome vector from 
# getStudentFeatures, fit a logistic regression model and return
# the list of coefficients, default sorted by absolute value

# Params: sort = ["pos", "neg", "abs"] to sort by largest positive, negative, or
# absolute value of coefficients
def get_coefs(X_train, X_test, y_train, y_test, sort = "abs"):
    model = LogisticRegression(solver="lbfgs")
    model.fit(X_train, y_train)
    train_score = model.score(X_train,y_train) 
    test_score = model.score(X_test,y_test)
    print("training accuracy: %s" % train_score)
    print("test accuracy: %s" % test_score)
    coefs = [(X_test.columns[i], model.coef_[0][i]) for i in range(len(X_test.columns))]
    if sort:
        if sort == "pos":
            coefs = sorted(coefs, key = lambda x: x[1], reverse = True)
        if sort == "neg":
            coefs = sorted(coefs, key = lambda x: x[1], reverse = False)
        if sort == "abs":
            coefs = sorted(coefs, key = lambda x: abs(x[1]), reverse = True)
    return coefs

In [13]:
def get_and_split_data(course_name, test_quarters):
    dataset = getStudentFeatures(course_name)
    dataset_grades = dataset.loc[dataset['strm'] < 1204.0] #For now, dropping this quarter
    dataset_train = dataset_grades.loc[~dataset_grades['strm'].isin(test_quarters)]
    dataset_test = dataset_grades.loc[dataset_grades['strm'].isin(test_quarters)]

    dataset_train = dataset_train.drop(["strm", "emplid"], axis=1)
    dataset_test = dataset_test.drop(["strm", "emplid"], axis=1)

    X_train = dataset_train.drop(["y"], axis=1)
    y_train = dataset_train["y"]
    X_test = dataset_test.drop(["y"], axis=1)
    y_test = dataset_test["y"]
    return X_train, X_test, y_train, y_test

In [14]:
X_train, X_test, y_train, y_test = get_and_split_data('CS110', [1194.0, 1196.0])
CS110coefs = get_coefs(X_train, X_test, y_train, y_test, 'abs')
CS110coefs[:100]

/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:252: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/ipykernel_launc

training accuracy: 0.7722222222222223
test accuracy: 0.7096774193548387


[('PSYCH90', 1.284760187903786),
 ('CS107', 1.2544970705223581),
 ('PHYSICS65', 1.1280294054963125),
 ('CS246', 1.0736717114483056),
 ('OSPKYOTO40M', 1.0468523700859078),
 ('MS&E221', 1.029692645071922),
 ('STATS60', -0.9993673126004036),
 ('MATH42', 0.9600855884867928),
 ('ME70', 0.9148508594440135),
 ('CS376', 0.9111228207863397),
 ('STATS202', -0.90085957191418),
 ('PHIL102', -0.8798421726187122),
 ('STATS200', 0.8789148414181183),
 ('MATH171', 0.8180620060606153),
 ('CS371', 0.8135370919141545),
 ('CS210A', -0.8115270830766116),
 ('PHYSICS63', -0.8041617828382792),
 ('PHYSICS25', 0.79799033760813),
 ('CS168', -0.782478414887705),
 ('CS107E', 0.7799476396263596),
 ('EE180', 0.7668643291908809),
 ('PHIL170', -0.7586771856210998),
 ('PHIL1', -0.7435933283216568),
 ('PUBLPOL104', 0.7434445984754675),
 ('CS166', 0.7361392144107692),
 ('PHYSICS105', -0.72770482090079),
 ('PHIL150', 0.7175996601790574),
 ('CHEM131', 0.7141192687613139),
 ('ENGR50', -0.7098964322423076),
 ('CS261', 0.70535

## LASSO logistic regression to reduce nonzero coefficients

In [15]:
def one_course_lasso(course_name, test_quarters = [1194.0, 1196.0]):
    X_train, X_test, y_train, y_test = get_and_split_data(course_name, test_quarters)
    for i in [10, 3, 1, 0.3, 0.1, 0.03, 0.01]:
      model = LogisticRegression(penalty="l1", solver="saga", C=i, max_iter=1000) 
      model.fit(X_train, y_train)
      train_score = model.score(X_train, y_train)
      test_score = model.score(X_test, y_test)
      coefs = [(X_train.columns[i], model.coef_[0][i]) for i in range(len(X_train.columns))]
      coefs_pos = sorted(coefs, key = lambda x: x[1], reverse = True)
      coefs_neg = sorted(coefs, key = lambda x: x[1], reverse = False)
      coefs_abs = sorted(coefs, key = lambda x: abs(x[1]), reverse = True)
      coefs_nonzero = sum([x[1] != 0 for x in coefs])
      coefs_zero = sum(x[1] == 0 for x in coefs)
      print("\n***** C = %f *****" % i)
      print("training accuracy: %s" % train_score)
      print("test accuracy: %s" % test_score)
      print("Number of nonzero/zero coefficients: %d/%d" % (coefs_nonzero, coefs_zero))
      print("Largest absolute coefficients:")
      pprint(coefs_abs[:3])
      print("Largest positive coefficients:")
      pprint(coefs_pos[:3])
      print("Largest negative coefficients:")
      pprint(coefs_neg[:3])

In [16]:
one_course_lasso("CS221")

/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:252: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/ipykernel_launc


***** C = 10.000000 *****
training accuracy: 0.8155416012558869
test accuracy: 0.7674418604651163
Number of nonzero/zero coefficients: 253/398
Largest absolute coefficients:
[('PSYCH146', -7.11326705394233),
 ('CS211', -5.815805830524613),
 ('EE169', -5.743094962324382)]
Largest positive coefficients:
[('CS131', 5.080140553544188),
 ('CS210B', 5.002118119900721),
 ('BIO42', 4.8426600050647375)]
Largest negative coefficients:
[('PSYCH146', -7.11326705394233),
 ('CS211', -5.815805830524613),
 ('EE169', -5.743094962324382)]

***** C = 3.000000 *****
training accuracy: 0.7990580847723705
test accuracy: 0.7674418604651163
Number of nonzero/zero coefficients: 190/461
Largest absolute coefficients:
[('CS211', -4.1230447275192255),
 ('EE169', -3.2139320176651966),
 ('CS131', 3.032048877771225)]
Largest positive coefficients:
[('CS131', 3.032048877771225),
 ('PHIL187', 2.9594584667151107),
 ('CS144', 2.363927833882222)]
Largest negative coefficients:
[('CS211', -4.1230447275192255),
 ('EE169',

In [17]:
one_course_lasso("CS229")

/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:252: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/ipykernel_launc


***** C = 10.000000 *****
training accuracy: 0.8544668587896254
test accuracy: 0.7972972972972973
Number of nonzero/zero coefficients: 222/415
Largest absolute coefficients:
[('PSYCH120', 8.0600511630798),
 ('EE278', 6.950252280262135),
 ('PSYCH90', -6.729250983238644)]
Largest positive coefficients:
[('PSYCH120', 8.0600511630798),
 ('EE278', 6.950252280262135),
 ('MS&E211', 4.220430385686788)]
Largest negative coefficients:
[('PSYCH90', -6.729250983238644),
 ('PSYCH164', -6.697705748258394),
 ('MS&E262', -6.475177591810779)]

***** C = 3.000000 *****
training accuracy: 0.8386167146974063
test accuracy: 0.7702702702702703
Number of nonzero/zero coefficients: 157/480
Largest absolute coefficients:
[('CHEM135', -3.472044051522395),
 ('PSYCH90', -3.370033807743413),
 ('LINGUIST1', 2.8816622526953797)]
Largest positive coefficients:
[('LINGUIST1', 2.8816622526953797),
 ('PSYCH120', 2.279210596161708),
 ('STATS60', 2.2053723748392904)]
Largest negative coefficients:
[('CHEM135', -3.4720440